In [2]:
#%%

#%%
# Use this file to check that cerebro has loaded the data
# Setup
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import backtrader as bt
import os
from datetime import datetime
from datetime import timedelta
from datetime import date
import yfinance as yf
import pandas as pd
import numpy as np
from tqdm import tqdm
import pyfolio as pf
from Strategies import *

from matplotlib import pyplot as plt
import pickle
PATH_TO_SEC_DATA=os.environ['PATH_TO_SEC_DATA']

# Unpickle tickers
print("Unpickling Tickers")
with open(f'tickers', "rb") as input_file:
    tickers = pickle.load(input_file)

# When to start
initial_date = datetime(2010,1,1)
last_date = datetime(2021,9,1)
# Adds commissions
# 0.1% ... divide by 100 to remove the %

# Unpickle
#%%
load_again = True
use_all_tickers = False

if use_all_tickers:
    df = pd.read_csv(f"{PATH_TO_SEC_DATA}\\cik_ticker.csv")
    tickers = df.ticker
    tickers = list(set(tickers))



if load_again:
#tickers = tqdm(tickers)
    not_added = 0
    message = ""
    cerebro = bt.Cerebro()
   
    for ticker in tickers:
        #tickers.set_description(f"Adding {ticker} - {not_added} tickers failed to meet Cerebro. Last Error : {message} ")
        #tickers.refresh() # to show immediately the update
        try:
            df = yf.download(ticker.upper(), initial_date.strftime("%Y-%m-%d"), last_date.strftime("%Y-%m-%d"))
            if len(df):
                data = bt.feeds.PandasData(dataname=df)
                cerebro.adddata(data, name = ticker)
            else:
                not_added = not_added + 1
        except Exception as e:
            message = e
            not_added = not_added + 1

    #%%

    print("Pickling Cerebro")
    filename = f'cerebro_St'
    outfile = open(filename,'wb')
    pickle.dump(cerebro,outfile)
    outfile.close()

#cerebro.broker.setcommission(commission=0.001)
with open("cerebro_St", "rb") as f:
    cerebro = pickle.load(f)

cerebro.addstrategy(St)
cerebro.broker.setcash(1000000.0)
# To analyze data
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')


#%%
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
results =cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


# %%


Unpickling Tickers
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[**************

KeyboardInterrupt: 

In [ ]:
#Plot against a benchmark
strat = results[0]
pyfoliozer = strat.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()
df = yf.download("^GSPC", initial_date.strftime("%Y-%m-%d"), last_date.strftime("%Y-%m-%d"))
df['ret'] = df['Adj Close'].pct_change()
df.index 
df.index = df.index.tz_localize('UTC')
benchmark = df.ret
benchmark.index.names = ['index']
df = pd.merge(returns, benchmark, left_index=True, right_index=True)
output = pf.create_full_tear_sheet(df['return'], benchmark_rets =df.ret)
